### Binary Search Tree, Red Black Tree and AVL Tree

In [21]:
class TreeNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.parent = None  

class BinarySearchTree:
    def __init__(self):
        self.root = None

    def insert(self, newNode):
        parent = None
        curr = self.root
        while curr is not None:
            parent = curr
            if newNode.key < curr.key:
                curr = curr.left
            else:
                curr = curr.right
        newNode.parent = parent
        if parent is None: 
            self.root = newNode
        elif newNode.key < parent.key:
            parent.left = newNode
        else:
            parent.right = newNode

    def search(self, node, key_lookup):
        if node is None or key_lookup == node.key:
            return node
        if key_lookup < node.key:
            return self.search(node.left, key_lookup)
        else:
            return self.search(node.right, key_lookup)

    def minimum(self, node):
        while node.left is not None:
            node = node.left
        return node

    def transplant(self, replace_subtree, subtree_rooted):
        if replace_subtree.parent is None:
            self.root = subtree_rooted
        elif replace_subtree == replace_subtree.parent.left:
            replace_subtree.parent.left = subtree_rooted
        else:
            replace_subtree.parent.right = subtree_rooted
        if subtree_rooted is not None:
            subtree_rooted.parent = replace_subtree.parent

    def delete(self, delete_node):
        if delete_node.left is None:
            self.transplant(delete_node, delete_node.right)
        elif delete_node.right is None:
            self.transplant(delete_node, delete_node.left)
        else:
            sucessorOfz = self.minimum(delete_node.right)
            if sucessorOfz.parent != delete_node:
                self.transplant(sucessorOfz, sucessorOfz.right)
                sucessorOfz.right = delete_node.right
                sucessorOfz.right.parent = sucessorOfz
            self.transplant(delete_node, sucessorOfz)
            sucessorOfz.left = delete_node.left
            sucessorOfz.left.parent = sucessorOfz

    def inorderT(self, node, res):
        if node:
            self.inorderT(node.left, res)
            res.append(node.key)
            self.inorderT(node.right, res)

bt = BinarySearchTree()
nodes = [TreeNode(10), TreeNode(20), TreeNode(30), TreeNode(40), TreeNode(50)]
for i in nodes:
    bt.insert(i)

inorderRes = []
bt.inorderT(bt.root, inorderRes)
print("Inorder traversal after insertion:",inorderRes)

print("Search 10:", bt.search(bt.root, 10) is not None)  # It should print True
print("Search 30:", bt.search(bt.root, 30) is not None)  # It should print True
print("Search 80:", bt.search(bt.root, 80) is not None)  # It should print False
print("Search 120:", bt.search(bt.root, 120) is not None)  # It should print False


bt.delete(bt.search(bt.root, 10))  
inorderRes = []
bt.inorderT(bt.root, inorderRes)
print("Inorder traversal after deleting 10:", inorderRes)


Inorder traversal after insertion: [10, 20, 30, 40, 50]
Search 10: True
Search 30: True
Search 80: False
Search 120: False
Inorder traversal after deleting 10: [20, 30, 40, 50]


In [20]:
class Node:
    def __init__(self, key, color):
        self.key = key
        self.color = color 
        self.left = None
        self.right = None
        self.parent = None


class RedBlackTree:
    def __init__(self):
        self.NIL = Node(None, 'BLACK')  
        self.root = self.NIL

    def left_rotate(self, curr):
        parentNew = curr.right
        curr.right = parentNew.left
        if parentNew.left != self.NIL:
            parentNew.left.parent = curr
        parentNew.parent = curr.parent
        if curr.parent is None:
            self.root = parentNew
        elif curr == curr.parent.left:
            curr.parent.left = parentNew
        else:
            curr.parent.right = parentNew
        parentNew.left = curr
        curr.parent = parentNew

    def right_rotate(self, curr):
        parentNew = curr.left
        curr.left = parentNew.right
        if parentNew.right != self.NIL:
            parentNew.right.parent = curr
        parentNew.parent = curr.parent
        if curr.parent is None:
            self.root = parentNew
        elif curr == curr.parent.right:
            curr.parent.right = parentNew
        else:
            curr.parent.left = parentNew
        parentNew.right = curr
        curr.parent = parentNew

    def redBlack_insert(self, key):
        nodeNew = Node(key, 'RED')
        nodeNew.left = self.NIL
        nodeNew.right = self.NIL
        pNode = None
        curr = self.root

        while curr != self.NIL:
            pNode = curr
            if nodeNew.key < curr.key:
                curr = curr.left
            else:
                curr = curr.right

        nodeNew.parent = pNode
        if pNode is None:
            self.root = nodeNew
        elif nodeNew.key < pNode.key:
            pNode.left = nodeNew
        else:
            pNode.right = nodeNew

        self.redBlack_insert_fixup(nodeNew)

    def redBlack_insert_fixup(self, nodeNew):
        while nodeNew.parent and nodeNew.parent.color == 'RED':
            if nodeNew.parent == nodeNew.parent.parent.left:
                node1 = nodeNew.parent.parent.right
                if node1.color == 'RED':
                    nodeNew.parent.color = 'BLACK'
                    node1.color = 'BLACK'
                    nodeNew.parent.parent.color = 'RED'
                    nodeNew = nodeNew.parent.parent
                else:
                    if nodeNew == nodeNew.parent.right:
                        nodeNew = nodeNew.parent
                        self.left_rotate(nodeNew)
                    nodeNew.parent.color = 'BLACK'
                    nodeNew.parent.parent.color = 'RED'
                    self.right_rotate(nodeNew.parent.parent)
            else:
                node1 = nodeNew.parent.parent.left
                if node1.color == 'RED':
                    nodeNew.parent.color = 'BLACK'
                    node1.color = 'BLACK'
                    nodeNew.parent.parent.color = 'RED'
                    nodeNew = nodeNew.parent.parent
                else:
                    if nodeNew == nodeNew.parent.left:
                        nodeNew = nodeNew.parent
                        self.right_rotate(nodeNew)
                    nodeNew.parent.color = 'BLACK'
                    nodeNew.parent.parent.color = 'RED'
                    self.left_rotate(nodeNew.parent.parent)
        self.root.color = 'BLACK'

    def redBlack_delete(self, key):
        deletedNode = self.search(self.root, key)
        if deletedNode == self.NIL:
            return  
        
        givenColor = deletedNode.color
        if deletedNode.left == self.NIL:
            change_node = deletedNode.right
            self.transplant(deletedNode, deletedNode.right)
        elif deletedNode.right == self.NIL:
            change_node = deletedNode.left
            self.transplant(deletedNode, deletedNode.left)
        else:
            nextNode = self.minVal_node(deletedNode.right)
            givenColor = nextNode.color
            change_node = nextNode.right
            if nextNode.parent == deletedNode:
                change_node.parent = nextNode
            else:
                self.transplant(nextNode, nextNode.right)
                nextNode.right = deletedNode.right
                nextNode.right.parent = nextNode
            self.transplant(deletedNode, nextNode)
            nextNode.left = deletedNode.left
            nextNode.left.parent = nextNode
            nextNode.color = deletedNode.color

        if givenColor == 'BLACK':
            self.redBlack_delete_fixup(change_node)

    def transplant(self, prevNode, nodeNew):
        if prevNode.parent is None:
            self.root = nodeNew
        elif prevNode == prevNode.parent.left:
            prevNode.parent.left = nodeNew
        else:
            prevNode.parent.right = nodeNew
        nodeNew.parent = prevNode.parent

    def redBlack_delete_fixup(self, change_node):
        while change_node != self.root and change_node.color == 'BLACK':
            if change_node == change_node.parent.left:
                connected_node = change_node.parent.right
                if connected_node.color == 'RED':
                    connected_node.color = 'BLACK'
                    change_node.parent.color = 'RED'
                    self.left_rotate(change_node.parent)
                    connected_node = change_node.parent.right
                if connected_node.left.color == 'BLACK' and connected_node.right.color == 'BLACK':
                    connected_node.color = 'RED'
                    change_node = change_node.parent
                else:
                    if connected_node.right.color == 'BLACK':
                        connected_node.left.color = 'BLACK'
                        connected_node.color = 'RED'
                        self.right_rotate(connected_node)
                        connected_node = change_node.parent.right
                    connected_node.color = change_node.parent.color
                    change_node.parent.color = 'BLACK'
                    connected_node.right.color = 'BLACK'
                    self.left_rotate(change_node.parent)
                    change_node = self.root
            else:
                connected_node = change_node.parent.left
                if connected_node.color == 'RED':
                    connected_node.color = 'BLACK'
                    change_node.parent.color = 'RED'
                    self.right_rotate(change_node.parent)
                    connected_node = change_node.parent.left
                if connected_node.right.color == 'BLACK' and connected_node.left.color == 'BLACK':
                    connected_node.color = 'RED'
                    change_node = change_node.parent
                else:
                    if connected_node.left.color == 'BLACK':
                        connected_node.right.color = 'BLACK'
                        connected_node.color = 'RED'
                        self.left_rotate(connected_node)
                        connected_node = change_node.parent.left
                    connected_node.color = change_node.parent.color
                    change_node.parent.color = 'BLACK'
                    connected_node.left.color = 'BLACK'
                    self.right_rotate(change_node.parent)
                    change_node = self.root
        change_node.color = 'BLACK'

    def minVal_node(self, node):
        while node.left != self.NIL:
            node = node.left
        return node

    def search(self, curr, key):
        if curr == self.NIL or key == curr.key:
            return curr
        if key < curr.key:
            return self.search(curr.left, key)
        return self.search(curr.right, key)

    def inorderTrans(self, node, result):
        if node != self.NIL:
            self.inorderTrans(node.left, result)
            result.append((node.key, node.color))
            self.inorderTrans(node.right, result)

rb = RedBlackTree()
rb.redBlack_insert(10)
rb.redBlack_insert(20)
rb.redBlack_insert(30)
rb.redBlack_insert(40)
rb.redBlack_insert(50)

result = []
rb.inorderTrans(rb.root, result)
print("Red-Black Tree before deletion")
print(result)

rb.redBlack_delete(10)  
result = []
rb.inorderTrans(rb.root, result)
print("Red-Black Tree after deleting 10:")
print(result)

rb.redBlack_delete(20) 
result = []
rb.inorderTrans(rb.root, result)
print("Red-Black Tree after deleting both 10 and 20:")
print(result)

Red-Black Tree before deletion
[(10, 'BLACK'), (20, 'BLACK'), (30, 'RED'), (40, 'BLACK'), (50, 'RED')]
Red-Black Tree after deleting 10:
[(20, 'BLACK'), (30, 'RED'), (40, 'BLACK'), (50, 'BLACK')]
Red-Black Tree after deleting both 10 and 20:
[(30, 'BLACK'), (40, 'BLACK'), (50, 'BLACK')]
